# Navigation

<a href = "#A">Detailed Example</a>  
<a href = "#B">More Examples</a>  
<br>

## <font color='hotpink'>BQuant Application - Median EPS Change by Sector</font>  <a name="A"></a>

#### Initiate BQL

In [1]:
#import BQL library
import bql
#instantiate bq as the BQL service object
bq = bql.Service()

#### Function to run BQL query & return a dataframe

In [2]:
def pull_med_eps_chg(index_name, months_back):
    query = """
    get(
      avg(group(pct_chg(is_eps(fa_period_reference=2020Y, fa_adjusted=Y, dates=range(-{}m,0m),fill=PREV)),GICS_SECTOR_NAME()))
      as #pct_chg
    ) 
    for(MEMBERS('{}'))
    """.format(months_back, index_name)
    resp = bq.execute(query)
    df = resp[0].df()
    return df[[df.columns[-1]]]

#### Visualization Libaries

In [3]:
#import libraries
import ipywidgets as widgets
import bqviz as bqv
import bqplot as bqp
import pandas as pd
import numpy as np

#### Functions to create/update charts

In [4]:
# def pull_med_eps_chg(index_name, months_back):
#     df = pd.DataFrame(np.random.randn(10))
#     return df

def construct_bar_plot(df):
    plot = bqv.BarPlot(df)
    return plot.show()

def construct_pretty_bar_plot(df):
    dataframe = df

    # Create scales
    scale_x = bqp.OrdinalScale()
    scale_y = bqp.LinearScale()

    # Create mark
    tooltip = bqp.Tooltip(fields=['y'], show_labels=False, formats=['.3f'])
    mark_bar = bqp.Bars(x=dataframe.index,
                        #y=dataframe['Value'],
                        y=dataframe[dataframe.columns[-1]],
                        scales={'x': scale_x, 'y': scale_y},
                        padding=0.1,
                        colors=['#1B84ED', '#CF7DFF', '#FF5A00', '#00D3D6',
                                '#008616', '#B31D83', '#FF1E3E', '#FF9E24',
                                '#63B2FF', '#30C030'],
                        tooltip=bqp.Tooltip(fields=['y'],
                                            show_labels=False,
                                            formats=['.3f']))

    # Create Axes
    axis_y = bqp.Axis(scale=scale_y,
                      orientation='vertical',
                      label='Y Label')
    axis_x = bqp.Axis(scale=scale_x,
                      tick_rotate=20,
                      tick_style={'text-anchor': 'start'})

    # Create Figure
    figure = bqp.Figure(marks=[mark_bar],
                        axes=[axis_x, axis_y],
                        title='Vertical bar plot',
                        title_style={'font-size': '22px'},
                        padding_y=0,
                        fig_margin={'top': 50, 'bottom': 60,
                                    'left': 60, 'right': 50},
                        layout={'width': '100%',
                                'height': '500px'})

    def update_plot(dataframe):
        mark_bar.x = dataframe.index
        mark_bar.y = dataframe['Value']
        scale_x.domain = dataframe.index.tolist()

    # Display the figure
    return figure

def button_callback(btn):
    df = pull_med_eps_chg(index_picker.value, months_back_picker.value)
    #p = construct_bar_plot(df)
    p = construct_pretty_bar_plot(df)
    output_area.children=[p]
    

#### Create the GUI & connect button to functions

In [5]:
#GUI
index_picker = widgets.Dropdown(options=['SPX Index','IBOV Index', 'INDU Index'])
months_back_picker = widgets.IntSlider(description='Months Back', value=3,min=1,max=24)
button = widgets.Button(description='Click to Compute',button_style='success')

#button linked to function
button.on_click(button_callback)

# putting it all together
output_area = widgets.HBox([])
gui = widgets.VBox([index_picker, months_back_picker, button, output_area])
gui

<div style="text-align: right"><a href="#Navigation">Return to Top ↑</a></div>

## More Example Visualizations  <a name="B"></a>

In [16]:
import bqviz as bqv
from bqviz.sample_data import *

In [11]:
# For reference, display the first few rows of our dataframe
multidf.head()

,Benchmark,Sharpe Ratio,Sortino,Total Returns
2011-10-31,0.108338,0.007833,0.079679,0.382291
2011-11-30,0.109467,0.034100,-0.008712,0.375641
2011-12-31,0.127633,0.002486,-0.050788,0.380217
2012-01-31,0.156750,0.061014,-0.082341,0.378427
2012-02-29,0.174218,0.056343,0.056734,0.404381


In [12]:
# Pass the dataframe to LinePlot and apply Bloomberg styles
ex1 = bqv.LinePlot(multidf).set_style()

# Display the plot
ex1.show()

GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…

In [18]:
# Pass the dataframe to InteractiveLinePlot
ex18 = bqv.InteractiveLinePlot(multidf)

# Display the plot
ex18.show()

In [14]:
# Pass the dataframe to InteractiveScatterPlot
int_scatter = bqv.InteractiveScatterPlot(multidf)

# Display the plot
int_scatter.show()

<div style="text-align: right"><a href="#Navigation">Return to Top ↑</a></div>